## Word Embedding

Word Embeddings: Distributional vectors, also called word embeddings, are based on the so-called distributional hypothesis — words appearing within similar context possess similar meaning. Word embeddings are pre-trained on a task where the objective is to predict a word based on its context, typically using a shallow neural network. 

## Word2vec

Document was a vector in TFIDF but in world2vec ,it is a matrix becuase we convert in onehotencode to all words in a a document.
what is the relationship between words.Can we solve semantic like relationship in words by TDIDF.I think no. We can do this with the help of word2vec.
what does it do. Assign number to each words. and these numbers have meaning. We need to train a model word2vec to get this semantic relationship. It try to find the neighbourhood .
We find the probability of the word being nearby word.
what is near or neighbourhood word in document. It measn just near word left or right in document. we use window size to represent near word. id window size is 1 it measn wr consider just left and right 1 word and if windows size 2 then we conside near 2 words. 

word2vec embeddings: we use 2 methoths.
1. CBOW(contineous bag of word) : we predict the target word by using context words. as per below image.


<img src="https://raw.githubusercontent.com/teja/Machine_Learning/master/Images/cbow.PNG" width="440" height="240" align="left"/>

2. Skip-Gram : we predict the contexts word by using target word.It have 1 input,i output and 1 hidden layer. Input vector size is equal to no of unique words or no of vacabulary.Lets If we have 500 unique words and we put input word in the form of one hot encode then each word input size would be (500,1). we can select No of neurons as per our choice and output of hidden layer wll be feed into output layer. No of output from output layer is same as expected no of unique words. Output will be probability of neighbourhood words , It measn high probability measn near by word and low probability measn non-neighbour word. 
Lets if we have windows size of 5 then we will get total 10 words which will have high probability and remaining will have very low probability. Word2Vec uses Softmax as the last layer and Cross-Entropy loss. Now, the problem with Softmax is that the gradient is dependent on the summation across all classes.So when we try to train the such model where we only get very less amount of high probability then model will try to learn the always low probability. But Our Model should predict the positive probability to get the neighbour word. So we use negative sampling so that our model can predict the positive probability aslo. In negative sampling we only update the few weights. measn weights corresponding to positive outputs and very small number of weights of negative outputs. After traiining we discard the output layer and for each input word we get N words where N is the no of neurons. Also N is called as embedding size.

<img src="https://raw.githubusercontent.com/atulpatelDS/Machine_Learning/master/Images/skip-gram.PNG" width="440" height="240" align="left"/>

A word embedding model is a model that can provide numerical vectors for a given word. Using the Gensim’s downloader API, we can download pre-built word embedding models like word2vec, fasttext, GloVe and ConceptNet. These are built on large corpuses of commonly occurring text data such as wikipedia, google news etc.

However, if we are working in a specialized niche such as technical documents, we may not able to get word embeddings for all the words. So, in such cases its desirable to train our own model.

Gensim’s Word2Vec implementation let’s train our own word embedding model for a given corpus.

In [0]:
import gensim
import pandas as pd
import re,string
import numpy as np


In [0]:
dataset = pd.read_csv("https://raw.githubusercontent.com/atulpatelDS/Data_Files/master/Bag_of_Words/word2vec_nlp/unlabeledTrainData.tsv.zip",
                      header=0,delimiter="\t",quoting=3)

In [3]:
dataset.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [4]:
dataset.shape

(50000, 2)

In [5]:
dataset["review"][0]

'"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \\"Hey, let\'s pool our money together and make a really bad movie!\\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film\'s release. Life\'s like that."'

In [0]:
## We need to clean the string data before vectorization
def clean_string(string):
  try:
    str=re.sub(r"^https?:\/\/<>.*[\r\n]*", '',string, flags=re.MULTILINE)
    str=re.sub(r"[^A-Za-z]", " ", string)
    words = str.strip().lower().split()
    words = [w for w in words if len(w)>1]
    return " ".join(words)
  except:
    return ""

In [7]:
## Clean the review data
dataset["review_clean"] = dataset["review"].apply(clean_string)
dataset.head() 

,id,review,review_clean
0,"""9999_0""","""Watching Time Chasers, it obvious that it was...",watching time chasers it obvious that it was m...
1,"""45057_0""","""I saw this film about 20 years ago and rememb...",saw this film about years ago and remember it ...
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B...",minor spoilers br br in new york joan barnard ...
3,"""7161_0""","""I went to see this film with a great deal of ...",went to see this film with great deal of excit...
4,"""43971_0""","""Yes, I agree with everyone on this site this ...",yes agree with everyone on this site this movi...


In [0]:
## Now Convert the review_clean column into words list document
document = []
for word in dataset["review_clean"]:
  document.append(word.split(" "))

In [10]:
print(len(document))
print(document[0])

50000
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'bunch', 'of', 'friends', 'maybe', 'they', 'were', 'sitting', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'said', 'hey', 'let', 'pool', 'our', 'money', 'together', 'and', 'make', 'really', 'bad', 'movie', 'or', 'something', 'like', 'that', 'what', 'ever', 'they', 'said', 'they', 'still', 'ended', 'up', 'making', 'really', 'bad', 'movie', 'dull', 'story', 'bad', 'script', 'lame', 'acting', 'poor', 'cinematography', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', 'etc', 'all', 'corners', 'were', 'cut', 'except', 'the', 'one', 'that', 'would', 'have', 'prevented', 'this', 'film', 'release', 'life', 'like', 'that']


In [0]:
## Build the Model
model = gensim.models.Word2Vec(document, ## sentences--No of words in documents in the form of list
                               min_count=8, ## take only those words which have frequency greater or equal to 10
                               workers=4, ## No of CPU
                               size=50, ## Embeddeing Size or No of neurons in the hidden layer
                               window=5, ##max distance between target and actual words --like left and right distance
                               iter=10) 

In [14]:
## Now our model in trainned and lets display How many words we have in our trainned Model
model.wv.vectors.shape
## Model has unique words --31536(size of vocabulary) and each word represented by 50

(31536, 50)

In [0]:
## Lets display the Vocabulary of the Model- unique words-31536
#model.wv.vocab

In [16]:
## Lets Display the Embedding of any unique word
model.wv["watching"]  ## can see total size is 50

array([-1.7483547 , -1.078792  , -2.413652  ,  2.932797  , -2.697756  ,
        1.0881611 , -3.8424175 , -0.19514327, -3.9948156 , -0.5793845 ,
       -3.6546783 , -5.3106675 ,  1.7451113 , -0.92208755, -1.1085382 ,
        1.9618555 ,  3.200684  ,  3.885689  , -1.7869713 , -4.0070143 ,
        1.4089153 , -2.408474  , -2.1207728 ,  1.7571901 ,  4.6971707 ,
        2.946527  , -2.180171  ,  0.11515046,  1.1930338 ,  0.28849223,
       -0.931327  ,  1.740871  , -0.65453786, -3.2568872 ,  1.997625  ,
        1.0446408 ,  1.4336432 ,  2.3780184 , -0.4502919 ,  2.1203218 ,
        0.7588038 ,  2.4503772 , -2.298043  , -2.2910674 ,  0.91410625,
       -1.8854061 , -2.6668775 ,  0.9612222 ,  3.5591452 ,  1.4377389 ],
      dtype=float32)

In [17]:
##  Lets display words which have similar meaning
model.wv.most_similar("perfectly")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('brilliantly', 0.8093279600143433),
 ('superbly', 0.7993444204330444),
 ('wonderfully', 0.7939373254776001),
 ('beautifully', 0.7793017625808716),
 ('nicely', 0.756568193435669),
 ('expertly', 0.7182832956314087),
 ('excellently', 0.6974278688430786),
 ('admirably', 0.6648328304290771),
 ('perfection', 0.6566228866577148),
 ('deftly', 0.6484870910644531)]

We can see we got the most similar words related to perfectly.We only just trained the model and use this.How it is showing such a good result. If we have very large dataset we can even get good result.

In [0]:
## Lets display those words which are not similar to "perfectly"
model.wv.doesnt_match(["fire","water","sea","car"])

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'sea'

We can see that we got output "kitchen", which is not a similar to other words.

In [0]:
## Save the model
model.save("word2vec-movie-imdb")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
## Lets load model and test
trainned_model = gensim.models.Word2Vec.load("word2vec-movie-imdb")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


1. Equation king + man = queen + ?
2. In this case there may not be enough data for this equation

In [0]:
trainned_model.most_similar(positive=["king","queen"],negative=["man"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('askwith', 0.6453505754470825),
 ('temple', 0.5604738593101501),
 ('hood', 0.5595288276672363),
 ('notre', 0.5591071844100952),
 ('mansfield', 0.5518171191215515),
 ('dallas', 0.5481864809989929),
 ('genie', 0.5472075939178467),
 ('zeta', 0.5395662784576416),
 ('indiana', 0.5339698195457458),
 ('kings', 0.5313084721565247)]

AS we see that we dont have enough dataset so we did not get expected result. But it is still showing most similar words corresponding to men.

We have trained and saved a Word2Vec model for our document. However, when a new dataset comes, you want to update the model so as to account for new words.

One of the challenges with word embedding methods is when we want to obtain vector representations for phrases such as “hot potato” or “Boston Globe”. We can’t just simply combine the individual word vector representations since these phrases don’t represent the combination of meaning of the individual words. And it gets even more complicated when longer phrases and sentences are considered.

The other limitation with the word2vec models is that the use of smaller window sizes produce similar embeddings for contrasting words such as “good” and “bad”, which is not desirable especially for tasks where this differentiation is important such as sentiment analysis. Another caveat of word embeddings is that they are dependent on the application in which they are used. Re-training task specific embeddings for every new task is an explored option but this is usually computationally expensive and can be more efficiently addressed using negative sampling. Word2vec models also suffer from other problems such as not taking into account polysemy and other biases that may surface from the training data.

The limits of approaches such as Word2Vec are also important in helping us understand the future trends of NLP research. They set a benchmark for all future research. So where did they come up short?

There is only one word embedding per word, i.e. word embeddings can only store one vector for each word. So “bank” only had one meaning for “I lodged money in the bank” and “there is a nice bench on the bank of the river”
1. They are difficult to train on large datasets
2. You couldn’t fine tune them. To tailor them to your domain you need to train them from scratch
3. They are not really a deep neural network. They are trained on a neural network with one hidden layer.
